In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import cross_val_score,KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets
from sklearn import svm
import scipy.stats as stats
import statistics
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB


pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', None)

In [ ]:
#import the data sets
gsdb_unclean=pd.read_csv("C:\\Users\\carol\\Desktop\\School\\Ryerson\\Final Project CIND 820\\datasets\\GSDB_V2.csv")
hum_rights_unclean=pd.read_csv("C:\\Users\\carol\\Desktop\\School\\Ryerson\\Final Project CIND 820\\datasets\\human rights violations2.csv")
corruption=pd.read_csv("C:\\Users\\carol\\Desktop\\School\\Ryerson\\Final Project CIND 820\\datasets\\corruption.csv")
QOLI=pd.read_csv("C:\\Users\\carol\\Desktop\\School\\Ryerson\\Final Project CIND 820\\datasets\\physical-quality-of-life-index-scale-0-100-1980–2010.csv")
sanctions=pd.read_csv("C:\\Users\\carol\\Desktop\\School\\Ryerson\\Final Project CIND 820\\datasets\\Sanctions.csv")

In [ ]:
#Information about the databases:

#GSDB: 
#
#
#

#hum_rights
#
#
#

#corruption
#
#
#

#Quality of Life Index (QOLI)
#
#
#



In [ ]:
#First look at the GSDB data - first we print the first five rows and column names to understand what is being presented.
gsdb_unclean.head()

In [ ]:
#take a look at the general information about the information in the database
gsdb_unclean.info()

In [ ]:
#perform description of the GSDB
gsdb_unclean.describe()

In [ ]:
#Begin cleaning up the database, first, remove unneeded columns - descr_trade is removed because it has many null values and 
# it specifically deals with trade sanctions which this analysis does not need to consider. Success is dropped as the success
# is attributed to self-declaration by the State and the purpose of this analysis is to determine whether the sanctions were
# in fact successful.
gsdb=gsdb_unclean
gsdb.pop("descr_trade")
gsdb.pop("case_id")
gsdb.pop("success")

In [ ]:
#The Human Rights data begins at 1981. Therefore, the GSDB should be limited to sanctions around that time frame. 
#Since the analysis is on the effect of sanctions on human rights violations, it would make sense to limit the GDSB data to 
#sanctions which occur or end within 2 years of sanctions. Therefore, sanctions ending prior to 1979 will not be included. 
#As the Human Rights data ends at 2011, sanctions imposed after that date will also not be included.
gsdb = gsdb[gsdb["end"]>1978]
gsdb = gsdb[gsdb["begin"]<2011]

In [ ]:
#Number of sanctions which are devoted to Human Rights issues
No_of_Sanct=len(gsdb_unclean.index)
HR_Sanct= len(gsdb.index)
print(HR_Sanct)
print(HR_Sanct/No_of_Sanct)
distribution= np.array([HR_Sanct, No_of_Sanct-HR_Sanct])
labels=["Human Rights Sanctions", "Non-Human Rights Sanctions"]
plt.pie(distribution, labels=labels)
plt.show()

In [ ]:
#types of sanctions for all sanctions - not just human rights
cols=["trade", "arms", "military", "financial", "travel", "other"]
types_sanctions=gsdb[cols].sum(axis=0)
types_sanctions.plot(kind="bar")
plt.show()

In [ ]:
#The focus of this analysis are sanctions imposed for the reason of human rights. Therefore, all DO NOT contain human rights
#as an objective are to be removed.First we check what different objectives there are.
print(gsdb["objective"].unique())

In [ ]:
#Remove all rows with objectives which are not "human_rights"
gsdb = gsdb[gsdb["objective"].str.contains("human_rights")]

In [ ]:
#view the types of sanctions for specifically human rights violations
cols=["trade", "arms", "military", "financial", "travel", "other"]
types_sanctions=gsdb[cols].sum(axis=0)
types_sanctions.plot(kind="bar")
plt.show()

In [ ]:
#Determine how long the sanctions were in place, make this into a new column named time
gsdb["time"] = gsdb["end"]-gsdb["begin"]+1
#Show the frequency of the time that sanctions are active
gsdb['time'].value_counts().sort_index().plot(kind="bar")
plt.xlabel("Years")
plt.ylabel("Count")
plt.show()
print("The Median is:", statistics.median(gsdb["time"]))

In [ ]:
#break up the databases into numerical and categorical to facilitate the different analyses
gsdb_num=gsdb[["begin", "end", "trade", "arms", "military", "financial", "travel", "time"]]
gsdb_cat=gsdb[["sanctioned_state", "sanctioning_state"]]

In [ ]:
for i in gsdb_num.columns:
    plt.hist(gsdb_num[i])
    plt.title(i)
    plt.show()

In [ ]:
#look at beginner correlations to see if there is any correlation between the types of sanctions imposed
print(gsdb_num.corr())
sn.heatmap(gsdb_num.corr())

In [ ]:
#Review the unique Target States
countries=gsdb["sanctioned_state"].unique()
print(countries)

In [ ]:
#Determine how many times a State was sanctions - will bring this into the human rights database
sanct_freq=gsdb['sanctioned_state'].value_counts()
#Determine the distribution of the target States.
sanct_freq.sort_index().plot(kind="bar")
plt.show()

In [ ]:
#THE HUMAN RIGHTS DATASET

In [ ]:
hum_rights_unclean.head()

In [ ]:
hum_rights_unclean.info()

In [ ]:
hum_rights_unclean.describe()

In [ ]:
#Unnecessary columns: CIRI, COW, POLITY, UNCTRY, UNREG, UNSUBREG, as these are identifiers
#Remove OLD columns as they are variable calculated in an older way and are outdated. 
hum_rights_unclean.pop("CIRI")
hum_rights_unclean.pop("COW")
hum_rights_unclean.pop("POLITY")
hum_rights_unclean.pop("UNCTRY")
hum_rights_unclean.pop("UNREG")
hum_rights_unclean.pop("UNSUBREG")
hum_rights_unclean.pop("OLD_EMPINX")
hum_rights_unclean.pop("OLD_MOVE")
hum_rights_unclean.pop("OLD_RELFRE")

In [ ]:
#According to the data description, any unknown data is listed as -999, any data during periods of interregnum or interruption
#are listed at -77 or -66. Therefore, any data below ZERO cannot be relied upon. Changing such numbers to ZERO would imply that
#there are serious violations, which could be misleading. Therefore, it would be best to remove such data and not include it 
#in most of the analysis
hum_rights=hum_rights_unclean
cols=["PHYSINT","DISAP","KILL","POLPRIS","TORT","NEW_EMPINX","ASSN","FORMOV","DOMMOV","SPEECH","ELECSD","NEW_RELFRE","WORKER","WECON","WOPOL","WOSOC","INJUD"]

hum_rights[cols] = hum_rights[cols].mask(np.isclose(hum_rights[cols].values, -999.000000))
hum_rights[cols] = hum_rights[cols].mask(np.isclose(hum_rights[cols].values, -77.000000))
hum_rights[cols] = hum_rights[cols].mask(np.isclose(hum_rights[cols].values, -66.000000))


In [ ]:
#plot preliminary histograms for the numeric data
for i in hum_rights.columns:
    plt.hist(hum_rights[i])
    plt.title(i)
    plt.show()

In [ ]:
#Normalize the data in order to perform analysis on it.
hum_rights[cols]=(hum_rights[cols]-hum_rights[cols].min())/(hum_rights[cols].max()-hum_rights[cols].min())
hum_rights=hum_rights.round(3) #for readability

In [ ]:
#A column is created to provide a summary number of the Human Rights Score total
hum_rights["HR_Score"] = hum_rights[cols].sum(axis=1)

In [ ]:
#Calculate the change in violation score
hum_rights["PREVIOUS"]=hum_rights["HR_Score"].shift(1)
hum_rights["CHANGE"]=hum_rights["HR_Score"]-hum_rights["PREVIOUS"]
hum_rights["CHANGE"]=hum_rights["CHANGE"].round(3)

In [ ]:
#look at the histograms again following the normalization and addition of columns
for i in hum_rights.columns:
    plt.hist(hum_rights[i])
    plt.title(i)
    plt.show()

In [ ]:
#create a db for unsanctioned states to compare the results
unsanctioned=hum_rights
unsanctioned=unsanctioned[~unsanctioned["CTRY"].isin(countries)]
unsanctioned.head()

In [ ]:
for i in unsanctioned.columns:
    plt.hist(unsanctioned[i])
    plt.title(i)
    plt.show()

In [ ]:
#Not every country in the human rights database has been subject to sanctions. We will remove those unsanctioned countries 
countries_inc=gsdb["sanctioned_state"]
countries_HR=hum_rights["CTRY"].unique()
sanct_countries=set(countries_HR).difference(countries_inc)
SC=list(sanct_countries)
hum_rights=hum_rights[~hum_rights['CTRY'].str.contains('|'.join(SC))]

In [ ]:
hum_rights.head()

In [ ]:
hum_rights.info()

In [ ]:
hum_rights.describe()

In [ ]:
#look at the histograms again following the removal of unsanctioned states
for i in hum_rights.columns:
    plt.hist(hum_rights[i])
    plt.title(i)
    plt.show()

In [ ]:
#Checking the correlation of the variables within the human rights dataframe
print(hum_rights.corr())
sn.heatmap(hum_rights.corr())

In [ ]:
#The top countries which have been sanctioned
hum_rights.loc[hum_rights['HR_Score'] > 16]

In [ ]:
#QUALITY OF LIFE INDEX

In [ ]:
QOLI.head()

In [ ]:
QOLI.info() #There are some missing values, which should be filled with means of the scores on either side

In [ ]:
QOLI.describe()

In [ ]:
Country_col=list(QOLI["Country"])
QOLI.pop("Country")
#fill the NaN values with the means of a row
QOLI=QOLI.transpose().fillna(QOLI.mean(axis=1)).transpose()
QOLI.pop("1980")
QOLI["2011"]=QOLI["2010"]#since the dataframe does not extend to 2011, we assume that the scores given in 2010 are similar
#to those which would be awarded in 2011
QOLI.insert(loc=0, column="CTRY", value=Country_col)

In [ ]:
#merge the human rights data with the QOLI information
melted=pd.melt(QOLI, id_vars=["CTRY"], value_vars=["1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011"], var_name="YEAR", value_name="QOLI_Score")
melted["YEAR"]=melted["YEAR"].astype(int)
hum_rights=hum_rights.merge(melted, on=['CTRY', 'YEAR'], how='left')

In [ ]:
hum_rights["QOLI_Score"]=hum_rights["QOLI_Score"].replace('nan', pd.NA)
QOLI_chart=hum_rights["QOLI_Score"]
QOLI_chart.hist(bins=200)
plt.xlabel("QOLI Score")
plt.ylabel("Frequency")
plt.title("QOLI Scores of all States in Database")
plt.show()

In [ ]:
#Check if the the distribution was similar in one year
QOLI_chart=hum_rights["QOLI_Score"].where(hum_rights["YEAR"]==2011)
QOLI_chart.hist(bins=30)
plt.show()

In [ ]:
med=statistics.median(hum_rights["QOLI_Score"])
mean=hum_rights["QOLI_Score"].mean()
mode=statistics.mode(hum_rights["QOLI_Score"])
print("The median is:",med, "The mean is:", mean,"The mode is:", mode)

In [ ]:
#Look at the distribution of the QOLI score when there are no sanctions
QOLI_unsanct=hum_rights["QOLI_Score"].where(hum_rights["SANCTION"]=="N")
QOLI_unsanct.hist(bins=200)
plt.xlabel("QOLI Score")
plt.ylabel("Frequency")
plt.title("QOLI Scores of unsanctioned States")
plt.show()
med=statistics.median(QOLI_unsanct)
mean=QOLI_unsanct.mean()
mode=statistics.mode(QOLI_unsanct)
print("The median is:",med, "The mean is:", mean,"The mode is:", mode)

In [ ]:
QOLI_sanct=hum_rights["QOLI_Score"].where(hum_rights["SANCTION"]=="Y")
QOLI_sanct=QOLI_sanct.dropna()
QOLI_sanct.hist(bins=200)
plt.xlabel("QOLI Score")
plt.ylabel("Frequency")
plt.title("QOLI Scores of sanctioned States")
plt.show()
med=statistics.median(QOLI_sanct)
mean=QOLI_sanct.mean()
mode=statistics.mode(QOLI_sanct)
print("The median is:",med, "The mean is:", mean,"The mode is:", mode)

In [ ]:
QOLI_y1=hum_rights["QOLI_Score"].where(hum_rights["SANCTION"]=="1")
QOLI_y1=QOLI_y1.dropna()
QOLI_y1.hist(bins=50)
plt.show()
med=statistics.median(QOLI_y1)
mean=QOLI_y1.mean()
mode=statistics.mode(QOLI_y1)
print("The median is:", med, "The mean is:", mean,"The mode is:", mode)

In [ ]:
QOLI_y2=hum_rights["QOLI_Score"].where(hum_rights["SANCTION"]=="2")
QOLI_y2=QOLI_y2.dropna()
QOLI_y2.hist(bins=50)
plt.show()
med=statistics.median(QOLI_y2)
mean=QOLI_y2.mean()
mode=statistics.mode(QOLI_y2)
print("The median is:",med, "The mean is:", mean,"The mode is:", mode)

In [ ]:
QOLI_y3=hum_rights["QOLI_Score"].where(hum_rights["SANCTION"]=="3")
QOLI_y3=QOLI_y3.dropna()
QOLI_y3.hist(bins=50)
plt.show()
med=statistics.median(QOLI_y3)
mean=QOLI_y3.mean()
mode=statistics.mode(QOLI_y3)
print("The median is:",med, "The mean is:", mean,"The mode is:", mode)

In [ ]:
QOLI_y4=hum_rights["QOLI_Score"].where(hum_rights["SANCTION"]=="4")
QOLI_y4=QOLI_y4.dropna()
QOLI_y4.hist(bins=50)
plt.show()
med=statistics.median(QOLI_y4)
mean=QOLI_y4.mean()
mode=statistics.mode(QOLI_y4)
print("The median is:",med, "The mean is:", mean,"The mode is:", mode)

In [ ]:
QOLI_y5=hum_rights["QOLI_Score"].where(hum_rights["SANCTION"]=="5")
QOLI_y5=QOLI_y5.dropna()
QOLI_y5.hist(bins=50)
plt.show()
med=statistics.median(QOLI_y5)
mean=QOLI_y5.mean()
mode=statistics.mode(QOLI_y5)
print("The median is:",med, "The mean is:", mean,"The mode is:", mode)

In [ ]:
#Corruption Data

In [ ]:
#bring in the corruption data and alter it to work with the other databases so that it can be brought into the human rights database
corruption.rename(columns={"Jurisdiction":"CTRY"}, inplace=True)
melted_cor=pd.melt(corruption, id_vars=["CTRY"], value_vars=["1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011"], var_name="YEAR", value_name="Corruption_Score")
melted_cor["YEAR"]=melted_cor["YEAR"].astype(int)
hum_rights = hum_rights.replace('-', np.nan)
hum_rights=hum_rights.merge(melted_cor, on=['CTRY', 'YEAR'], how='left')

In [ ]:
corruption.head()

In [ ]:
corruption.info()

In [ ]:
corruption.describe()

In [ ]:
hum_rights = hum_rights.replace('-', np.nan)
hum_rights["Corruption_Score"]=hum_rights["Corruption_Score"].astype(float)

In [ ]:
#overall corruption
corruption_all=hum_rights["Corruption_Score"]
corruption_all.hist(bins=200)
plt.xlabel("Corruption Score")
plt.ylabel("Frequency")
plt.title("Overall Corruption Scores")
plt.show()

In [ ]:
med=np.nanmedian(corruption_all)
mean=corruption_all.mean()
mode=statistics.mode(corruption_all)
print("The median is", med, "The mean is", mean, "The mode is", mode)

In [ ]:
Corruption_no_sanct=hum_rights["Corruption_Score"].where(hum_rights["SANCTION"]=="N")
Corruption_no_sanct.hist(bins=200)
plt.show()

In [ ]:
med=np.nanmedian(Corruption_no_sanct)
mean=Corruption_no_sanct.mean()
mode=statistics.mode(Corruption_no_sanct)
print("The median is", med, "The mean is", mean, "The mode is", mode)

In [ ]:
Corruption_sanct=hum_rights["Corruption_Score"].where(hum_rights["SANCTION"]=="Y")
Corruption_sanct.hist(bins=200)
plt.show()

In [ ]:
med=np.nanmedian(Corruption_sanct)
mean=Corruption_sanct.mean()
mode=statistics.mode(Corruption_sanct)
print("The median is", med, "The mean is", mean, "The mode is", mode)

In [ ]:
Corruption_y1=hum_rights["Corruption_Score"].where(hum_rights["SANCTION"]=="1")
Corruption_y1.hist(bins=30)
plt.show()

In [ ]:
med=np.nanmedian(Corruption_y1)
mean=Corruption_y1.mean()
mode=statistics.mode(Corruption_y1)
print("The median is", med, "The mean is", mean, "The mode is", mode)

In [ ]:
Corruption_y2=hum_rights["Corruption_Score"].where(hum_rights["SANCTION"]=="2")
Corruption_y2.hist(bins=30)
plt.show()

In [ ]:
med=np.nanmedian(Corruption_y2)
mean=Corruption_y2.mean()
mode=statistics.mode(Corruption_y2)
print("The median is", med, "The mean is", mean, "The mode is", mode)

In [ ]:
Corruption_y3=hum_rights["Corruption_Score"].where(hum_rights["SANCTION"]=="3")
Corruption_y3.hist(bins=30)
plt.show()

In [ ]:
med=np.nanmedian(Corruption_y3)
mean=Corruption_y3.mean()
mode=statistics.mode(Corruption_y3)
print("The median is", med, "The mean is", mean, "The mode is", mode)

In [ ]:
Corruption_y4=hum_rights["Corruption_Score"].where(hum_rights["SANCTION"]=="4")
Corruption_y4.hist(bins=30)
plt.show()

In [ ]:
med=np.nanmedian(Corruption_y4)
mean=Corruption_y4.mean()
mode=statistics.mode(Corruption_y4)
print("The median is", med, "The mean is", mean, "The mode is", mode)

In [ ]:
Corruption_y5=hum_rights["Corruption_Score"].where(hum_rights["SANCTION"]=="5")
Corruption_y5.hist(bins=30)
plt.show()

In [ ]:
med=np.nanmedian(Corruption_y5)
mean=Corruption_y5.mean()
mode=statistics.mode(Corruption_y5)
print("The median is", med, "The mean is", mean, "The mode is", mode)

In [ ]:
#BRING IN A SUCCESS COLUMN TO hum_rights
keys=sanctions["Target"]
values=sanctions["Success"]
dictionary = dict(zip(keys, values))
hum_rights["Success"]= hum_rights["CTRY"].map(dictionary)

In [ ]:
#show the distribution of successful and failed sanctions
success=sanctions["Success"]
success.hist()
plt.xlabel("")
plt.show()

In [ ]:
hum_rights["Success"].value_counts()

In [ ]:
success=527
fail=496
distribution= np.array([success, fail])
labels=["Success", "Fail"]
plt.pie(distribution, labels=labels)
plt.title("Distribution of Success vs Failure")
plt.show()

In [ ]:
no_sanct=hum_rights[hum_rights["SANCTION"]=="N"]
under_sanct=hum_rights[hum_rights["SANCTION"]=="Y"]
year1=hum_rights[hum_rights["SANCTION"]=="1"]
year2=hum_rights[hum_rights["SANCTION"]=="2"]
year3=hum_rights[hum_rights["SANCTION"]=="3"]
year4=hum_rights[hum_rights["SANCTION"]=="4"]
year5=hum_rights[hum_rights["SANCTION"]=="5"]
num_no_sanct=len(no_sanct)
num_sanct=len(under_sanct)
num_year1=len(year1)
num_year2=len(year2)
num_year3=len(year3)
num_year4=len(year4)
num_year5=len(year5)

In [ ]:
#the distribution of sanctions, no sanctions and years after sanctions
instances=[num_no_sanct, num_sanct, num_year1, num_year2, num_year3, num_year4, num_year5]
x_inst=("no_sanct", "sanct", "year1", "year2", "year3", "year4", "year5")
x_position=np.arange(len(x_inst))
plt.bar(x_position, instances)
plt.xticks(x_position, x_inst)
plt.xlabel("Sanction Status")
plt.ylabel("Count")
plt.show()

In [ ]:
#Remove no sanctionsto see the more important distributions
instances=[num_sanct, num_year1, num_year2, num_year3, num_year4, num_year5]
x_inst=("sanct", "year1", "year2", "year3", "year4", "year5")
x_position=np.arange(len(x_inst))
plt.bar(x_position, instances)
plt.xticks(x_position, x_inst)
plt.show()

In [ ]:
no_sanct_mean=no_sanct["CHANGE"].mean()
under_sanct_mean=under_sanct["CHANGE"].mean()
year1_mean=year1["CHANGE"].mean()
year2_mean=year2["CHANGE"].mean()
year3_mean=year3["CHANGE"].mean()
year4_mean=year4["CHANGE"].mean()
year5_mean=year5["CHANGE"].mean()

In [ ]:
#demonstrate the average change at each status time
means=[no_sanct_mean, under_sanct_mean, year1_mean, year2_mean, year3_mean, year4_mean, year5_mean]
x_inst=("no_sanct","sanct", "year1", "year2", "year3", "year4", "year5")
x_position=np.arange(len(x_inst))
plt.bar(x_position, means)
plt.xticks(x_position, x_inst)
plt.show()

In [ ]:
#T-TESTS
#First, we plot the distributions of the rate of change for each time
no_sanct.hist(column="HR_Score", bins=200)
plt.xlabel("Human Rights Score")
plt.ylabel("Frequency")
plt.title("No Sanction Scores")
plt.show()

In [ ]:
under_sanct.hist(column="HR_Score", bins=200)
plt.xlabel("Human Rights Score")
plt.ylabel("Frequency")
plt.title("Sanctioned Scores")
plt.show()

In [ ]:
unsanctioned.hist(column="HR_Score", bins=200)
plt.xlabel("Human Rights Score")
plt.ylabel("Frequency")
plt.title("Never Sanctioned States")
plt.show()

In [ ]:
year1.hist(column="HR_Score", bins=50)
plt.show()

In [ ]:
year2.hist(column="HR_Score", bins=50)
plt.show()

In [ ]:
year3.hist(column="HR_Score", bins=50)
plt.show()

In [ ]:
year4.hist(column="HR_Score", bins=50)
plt.show()

In [ ]:
year5.hist(column="HR_Score", bins=50)
plt.show()

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'N'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == 'Y'])


In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'N'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '1'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'N'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '2'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'N'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '3'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'N'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '4'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'N'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '5'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'Y'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '1'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'Y'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '2'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'Y'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '3'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'Y'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '4'])

In [ ]:
stats.ttest_ind(hum_rights['HR_Score'][hum_rights['SANCTION'] == 'Y'],
                hum_rights['HR_Score'][hum_rights['SANCTION'] == '5'])

In [ ]:
#do p val tests for QOLI and Corruption!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
stats.ttest_ind(hum_rights['QOLI_Score'][hum_rights['Success']=="1"],
                hum_rights['QOLI_Score'][hum_rights["Success"]=="0"])

In [ ]:
stats.ttest_ind(hum_rights['Corruption_Score'][hum_rights['Success']=="1"],
                hum_rights['Corruption_Score'][hum_rights["Success"]=="0"])

In [ ]:
#Determining which variable to drop for Predictive analysis. Will create a new db to work off of to not 
matrix=hum_rights.corr()
hum_rights.corr()

In [ ]:
sn.heatmap(matrix, annot=True)
plt.show()

In [ ]:
#remove any columns that have a correlation with "Success" of less than 0.10.
df=hum_rights.drop(['YEAR', 'PHYSINT', 'DISAP','KILL','TORT','FORMOV','DOMMOV', 'WORKER','WECON','INJUD','PREVIOUS'], axis=1)

In [ ]:
matrix=df.corr()
df.corr()

In [ ]:
sn.heatmap(matrix, annot=True)
plt.show()

In [ ]:
#A new database was created to demonstrate the sanctions success rate 
sanctions.head()

In [ ]:
sanctions.info()

In [ ]:
sanctions.describe()

In [ ]:
sanctions.drop("Sanctioning_State", inplace=True, axis=1)
sanctions.drop("Year_Start", inplace=True, axis=1)
sanctions.drop("Year_End", inplace=True, axis=1)
sanctions_clean=sanctions.dropna(how="any", subset=["Success"])

In [ ]:
matrix=sanctions.corr()
sanctions.corr()

In [ ]:
sn.heatmap(matrix, annot=True)
plt.show()

In [ ]:
#For NaN in any Year columns, we will use the average.
sanctions["Year1"]=sanctions["Year1"].fillna(sanctions["Average"])
sanctions["Year2"]=sanctions["Year2"].fillna(sanctions["Average"])
sanctions["Year3"]=sanctions["Year3"].fillna(sanctions["Average"])
sanctions["Year4"]=sanctions["Year4"].fillna(sanctions["Average"])
sanctions["Year5"]=sanctions["Year5"].fillna(sanctions["Average"])

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
total_score=sanctions["Average"]
country=sanctions["Target"]

ax.bar(country,total_score)
plt.show()

In [ ]:
#For NaN values immediately before the sanctions and the average of the years prior to the sanctions
# we will look at the average rate of change in the other columns and apply that.
av_init_change=sanctions["Start_of_Sanct"]-sanctions["Before_Sanct"]
#remove the NaN
av_init_change=av_init_change.dropna()
x=statistics.mean(av_init_change)
print(x)

#Seems to indicate the on average, the human rights score was higher immediately before a sanction

In [ ]:
sanctions["Before_Sanct"]=sanctions["Before_Sanct"].fillna(sanctions["Start_of_Sanct"]+x)
sanctions["Before_Sanct_avg"]=sanctions["Before_Sanct_avg"].fillna(sanctions["Before_Sanct"])

In [ ]:
#PREDICTIVE MODELLING

In [ ]:
hum_rights.head()

In [ ]:
df=hum_rights
df.dropna()
df = df.reset_index()
df = df[df['Success'].notna()]

In [ ]:
#Decision tree - hum_rights database
cols=["PHYSINT","DISAP","KILL","POLPRIS","TORT","NEW_EMPINX","ASSN","FORMOV","DOMMOV","SPEECH","ELECSD","NEW_RELFRE","WORKER","WECON","WOPOL","WOSOC","INJUD","HR_Score","QOLI_Score","Corruption_Score"]
df[cols].astype(np.float32)
df[cols]= np.nan_to_num(df[cols].astype(np.float32))
X=df[cols]
Y=df["Success"]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

In [ ]:
clf_entropy=DecisionTreeClassifier(criterion="entropy", random_state=100, max_depth=15, 
                                   min_samples_leaf=5)
clf_entropy.fit(X_train, y_train)

In [ ]:
y_pred_en=clf_entropy.predict(X_test)
y_pred_en

In [ ]:
print(("Accuracy is"), accuracy_score(y_test, y_pred_en)*100)

In [ ]:
#Decision tree - hum_rights database - REMOVE UNCORRELATED COLUMNS
cols=["POLPRIS","NEW_EMPINX","ASSN","SPEECH","ELECSD","NEW_RELFRE","WOPOL","WOSOC","HR_Score","QOLI_Score","Corruption_Score"]
df[cols].astype(np.float32)
df[cols]= np.nan_to_num(df[cols].astype(np.float32))
X=df[cols]
Y=df["Success"]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=100)

In [ ]:
clf_entropy=DecisionTreeClassifier(criterion="entropy", random_state=100, max_depth=10, 
                                   min_samples_leaf=5)
clf_entropy.fit(X_train, y_train)

In [ ]:
y_pred_en=clf_entropy.predict(X_test)
y_pred_en

In [ ]:
print(("Accuracy is"), accuracy_score(y_test, y_pred_en)*100)

In [ ]:
sanctions.head()

In [ ]:
dfs=sanctions
dfs.dropna()
dfs = dfs.reset_index()
dfs = dfs[dfs['Success'].notna()]

In [ ]:
#Decision tree - hum_rights database
cols=["Years","Before_Sanct","Before_Sanct_avg","Start_of_Sanct","End_of_Sanct","During_avg","Year1","Year2","Year3","Year4","Year5","Average"]
dfs[cols].astype(np.float32)
dfs[cols]= np.nan_to_num(dfs[cols].astype(np.float32))
X=dfs[cols]
Y=dfs["Success"]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=100)

In [ ]:
clf_entropy=DecisionTreeClassifier(criterion="entropy", random_state=100, max_depth=10, 
                                   min_samples_leaf=5)
clf_entropy.fit(X_train, y_train)

In [ ]:
y_pred_en=clf_entropy.predict(X_test)
y_pred_en

In [ ]:
print(("Accuracy is"), accuracy_score(y_test, y_pred_en)*100)

In [ ]:
#NAIVE BAYES on Human Rights Database

In [ ]:
df3=hum_rights
df3.dropna()
df3 = df3.reset_index()
df3 = df3[df3['Success'].notna()]

In [ ]:
cols=["HR_Score","QOLI_Score","Corruption_Score"]
df3[cols].astype(np.float32)
df3[cols]= np.nan_to_num(df3[cols].astype(np.float32))
X=df3[cols]
Y=df3["Success"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
y_pred  =  classifier.predict(X_test)

In [ ]:
 y_pred  

In [ ]:
y_test

In [ ]:
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test,y_pred)
cm


In [ ]:
ac

In [ ]:
#Random Forest Classifier

In [ ]:
df4=hum_rights
df4.dropna()
df4 = df4.reset_index()
df4 = df4[df4['Success'].notna()]
cols=["NEW_EMPINX","WOSOC","HR_Score","QOLI_Score","Corruption_Score"]
df4[cols].astype(np.float32)
df4[cols]= np.nan_to_num(df4[cols].astype(np.float32))
X=df4[cols]
Y=df4["Success"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0)


rf=RandomForestClassifier(n_estimators=50)
rf.fit(X_train, y_train)
suc=rf.predict(X_test)

In [ ]:
print(suc)

In [ ]:
print(y_test)

In [ ]:
accuracy_score(y_test,suc)

In [ ]:
df5=hum_rights
df5.dropna()
df5 = df5.reset_index()
df5 = df5[df5['Success'].notna()]
cols=["NEW_EMPINX","WOSOC","HR_Score","QOLI_Score","Corruption_Score"]
df5[cols].astype(np.float32)
df5[cols]= np.nan_to_num(df5[cols].astype(np.float32))
X=df5[cols]
Y=df5["Success"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0)

In [ ]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_val_score(clf, X, Y, cv=20)
scores

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
#K-fold cross validation

In [ ]:
df6=hum_rights
df6.dropna()
df6 = df6.reset_index()
df6 = df6[df6['Success'].notna()]
#cols=["NEW_EMPINX","WOSOC","HR_Score","QOLI_Score","Corruption_Score"]
cols=["POLPRIS","NEW_EMPINX","ASSN","SPEECH","ELECSD","NEW_RELFRE","WOPOL","WOSOC","HR_Score","QOLI_Score","Corruption_Score"]
df6[cols].astype(np.float32)
df6[cols]= np.nan_to_num(df6[cols].astype(np.float32))
X=df6[cols]
Y=df6["Success"]
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0)

In [ ]:
k = 3
kf = model_selection.KFold(n_splits=k, random_state=None)
model = LogisticRegression(solver= 'liblinear')
 
result = cross_val_score(model , X, Y, cv = kf)
 
print("Avg accuracy: {}".format(result.mean()))